In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import tensorflow as tf
import tensorflow.contrib.eager as tfe
from PIL import Image
import io


/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passi

In [2]:
tfe.enable_eager_execution()


In [3]:
train_path='./data/bird2_train/'
test_path='./data/bird2_test/'

## Create Dataframe from CVAT XML

In [4]:
def create_df_xml(input1):
    annotation=ET.parse(f'../data/bird2_{input1}/{input1}_annotations.xml')
    root=annotation.getroot()
    df_cols=['root','filename','w','h']
    df_rows=[]
    for node in root.iter('image'):
        #print (list(node.attrib.values()))
        s_root=f'../data/bird2_{input1}/'
        s_name=list(node.attrib.values())[1]
        s_height=list(node.attrib.values())[3]
        s_width=list(node.attrib.values())[2]
         
        
    
        df_rows.append({'root':s_root,'filename': s_name,'w':s_width,'h':s_height,
                            })
        x1=pd.DataFrame(df_rows, columns = df_cols)
    df_cols=['x1','x2','y1','y2']
    df_rows=[]
    for node in root.iter('image'):
        for node1 in node:
            s_x1=list(node1.attrib.values())[2]
            s_x2=list(node1.attrib.values())[4]
            s_y1=list(node1.attrib.values())[3]
            s_y2=list(node1.attrib.values())[5]
   
            df_rows.append({
                            'x1':s_x1,'x2':s_x2,'y1':s_y1,'y2':s_y2})
        x2=pd.DataFrame(df_rows,columns=df_cols)
    x4= pd.concat((x1,x2),axis=1)    
    x4['w']=x4['w'].astype(int)
    x4['h']=x4['h'].astype(int)
    x4['x1']=x4['x1'].astype(float).round().astype(int)
    x4['x2']=x4['x2'].astype(float).round().astype(int)
    x4['y1']=x4['y1'].astype(float).round().astype(int)
    x4['y2']=x4['y2'].astype(float).round().astype(int) 
    return x4

In [5]:
test1=create_df_xml('test')
train1=create_df_xml('train')

In [6]:
train1.head()

,root,filename,w,h,x1,x2,y1,y2
0,../data/bird2_train/,Blue_Jay_00.jpg,600,391,102,445,52,333
1,../data/bird2_train/,Blue_Jay_001.jpg,960,640,403,835,72,435
2,../data/bird2_train/,Blue_Jay_00111.jpg,480,360,42,430,43,269
3,../data/bird2_train/,Blue_Jay_0031.jpg,570,380,257,487,60,309
4,../data/bird2_train/,Blue_Jay_00343.jpg,540,346,118,472,35,245


## Read TFRecord and write images to directory

In [7]:
raw_image_dataset = tf.data.TFRecordDataset('../data/bird2_train/birds.tfrecord')

In [8]:
image_feature_description = {
    'image/filename': tf.io.FixedLenFeature([], tf.string),
    'image/height': tf.io.FixedLenFeature([], tf.int64),
    'image/width': tf.io.FixedLenFeature([], tf.int64),
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    'image/object/bbox/xmin':tf.io.FixedLenFeature([], tf.float32),
    'image/object/bbox/xmax':tf.io.FixedLenFeature([], tf.float32),
    'image/object/bbox/ymin':tf.io.FixedLenFeature([], tf.float32),
    'image/object/bbox/ymax':tf.io.FixedLenFeature([], tf.float32)

}

In [9]:
def _parse_image_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

<DatasetV1Adapter shapes: {image/encoded: (), image/filename: (), image/height: (), image/object/bbox/xmax: (), image/object/bbox/xmin: (), image/object/bbox/ymax: (), image/object/bbox/ymin: (), image/width: ()}, types: {image/encoded: tf.string, image/filename: tf.string, image/height: tf.int64, image/object/bbox/xmax: tf.float32, image/object/bbox/xmin: tf.float32, image/object/bbox/ymax: tf.float32, image/object/bbox/ymin: tf.float32, image/width: tf.int64}>

In [10]:
df_cols=['root','filename','w','h','x1','x2','y1','y2']
df_rows=[]

for i,image_features in enumerate(parsed_image_dataset):
    
    ##grab data from our parsed TFRecord file
    s_root='../data/bird2_train/'
    image_raw = image_features['image/encoded'].numpy()
    image_height=image_features['image/height'].numpy()
    image_width=image_features['image/width'].numpy()
    image_x1=int(round(image_features['image/object/bbox/xmin'].numpy()*image_width))
    image_x2=int(round(image_features['image/object/bbox/xmax'].numpy()*image_width))
    image_y1=int(round(image_features['image/object/bbox/ymin'].numpy()*image_height))
    image_y2=int(round(image_features['image/object/bbox/ymax'].numpy()*image_height))
    filename=str(image_features['image/filename'].numpy())
    ##extract filename
    start = "b'"
    end = ".rf"
    s = filename
    filename=s[s.find(start)+len(start):s.rfind(end)]
    
    #write out and save image to our training folder
    #image = Image.open(io.BytesIO(image_raw))
    #image.save(f'../data/bird2_train/{filename}_{i}'+'.jpg', 'jpeg')
    filename=f'{filename}_{i}'
    #put data into dataframe
    df_rows.append({'root':s_root,'filename':filename+'.jpg','w':image_width,'h':image_height,'x1':image_x1,'x2':image_x2,
                    'y1':image_y1,'y2':image_y2
                            })
x4=pd.DataFrame(df_rows, columns = df_cols)


Instructions for updating:
Colocations handled automatically by placer.


In [13]:
print(x4.shape)
x4.head()

(428, 8)


,root,filename,w,h,x1,x2,y1,y2
0,../data/bird2_train/,Western_Meadowlark_0050_77864_jpg_0.jpg,500,400,126,396,110,272
1,../data/bird2_train/,Blue_Jay_0087_jpg_1.jpg,307,415,16,271,45,321
2,../data/bird2_train/,Western_Meadowlark_0112_78760_jpg_2.jpg,500,400,149,382,14,303
3,../data/bird2_train/,Horned_Grebe_0049_34779_jpg_3.jpg,500,333,125,441,55,262
4,../data/bird2_train/,Western_Meadowlark_0021_78841_jpg_4.jpg,333,500,109,216,56,151


In [14]:
train2=pd.concat((train1,x4),ignore_index=True)

In [15]:
print(train2.shape)
print(test1.shape)

(646, 8)
(102, 8)


In [17]:
train2.to_csv('../data/bird2_train/train2_df.csv')

In [18]:
test1.to_csv('../data/bird2_test/test1_df.csv')